In [1]:
import os
import tensorflow as tf
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose, UpSampling2D, concatenate, Input, Cropping2D
from keras.models import Model
from keras.optimizers import Adam
import math
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from skimage.io import imread
from keras.models import load_model
from keras.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K

Using TensorFlow backend.


## 1. Read Images

In [3]:
TRAIN_IMG_PATH = 'banco_dados_SB/train/imagens/'
TRAIN_LBL_PATH = 'banco_dados_SB/train/classes/'

TEST_IMG_PATH = 'banco_dados_SB/test/imagens/'
TEST_LBL_PATH = 'banco_dados_SB/test/classes/'

In [4]:
X_train = [imread(fname=TRAIN_IMG_PATH + img, as_grey=True) for img in os.listdir(TRAIN_IMG_PATH)]
y_train = [imread(fname=TRAIN_LBL_PATH + lbl)[:, :, 0] for lbl in os.listdir(TRAIN_LBL_PATH)]
X_test = [imread(fname=TEST_IMG_PATH + img, as_grey=True) for img in os.listdir(TEST_IMG_PATH)]
y_test = [imread(fname=TEST_LBL_PATH + lbl)[:, :, 0] for lbl in os.listdir(TEST_LBL_PATH)]

In [35]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 2. Criando o objeto Series e exportando para excel

In [6]:
obj = Series(os.listdir(TRAIN_IMG_PATH))

In [7]:
obj.to_excel('lista_imgs.xls')

In [7]:
obj2 = Series(os.listdir(TEST_IMG_PATH))
obj2.to_excel('teste2.xls')

## 3. importando excel e customizando

In [8]:
teste = pd.read_excel('lista_imgs.xls')

In [9]:
teste.head()

,0,TIPO
0,1.png,HFNI
1,2.png,HFNI
2,4.png,HFNI
3,5.png,HFNI
4,6.png,HFNI


In [11]:
teste['TIPO'].value_counts()

HFI     100
HFNI     98
HYP      84
N        56
Name: TIPO, dtype: int64

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

In [18]:
split = StratifiedShuffleSplit (n_splits=1, test_size=0.2, random_state = 42)
for train_index,validation_index in split.split(teste,teste['TIPO']):
    strat_train_set = teste.iloc[train_index]
    strat_valid_set = teste.iloc[validation_index]

In [19]:
print(teste['TIPO'].value_counts() /len(teste))
print('tamanho do set imagens:',len(teste))

HFI     0.295858
HFNI    0.289941
HYP     0.248521
N       0.165680
Name: TIPO, dtype: float64
tamanho do set imagens: 338


In [20]:
print(strat_train_set['TIPO'].value_counts() /len(strat_train_set))
print('tamanho de treino:',len(strat_train_set))

HFI     0.296296
HFNI    0.288889
HYP     0.248148
N       0.166667
Name: TIPO, dtype: float64
tamanho de treino: 270


In [21]:
print(strat_valid_set['TIPO'].value_counts() /len(strat_valid_set))
print('tamanho de validação:',len(strat_valid_set))

HFNI    0.294118
HFI     0.294118
HYP     0.250000
N       0.161765
Name: TIPO, dtype: float64
tamanho de validação: 68


In [22]:
strat_train_set.head()

,0,TIPO
238,295.png,N
102,122.png,HYP
82,99.png,HYP
156,189.png,HFI
254,315.png,HFNI


In [23]:
for set_ in (strat_train_set,strat_valid_set):
    set_.drop('TIPO',axis=1,inplace=True)

C:\Users\Tales\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
strat_train_set.head()

,0
238,295.png
102,122.png
82,99.png
156,189.png
254,315.png


In [29]:
list_train = list(strat_train_set[0])

In [30]:
list_valid = list(strat_valid_set[0])

In [38]:
np.save('lista_train',list_train)
np.save('lista_validation',list_valid)